In [30]:
from __future__ import print_function

from random import random
from numpy import array
from numpy import cumsum
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers import LSTM, Embedding
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional, Concatenate, concatenate, Dropout, Input, Merge, TimeDistributed, Flatten
from keras.models import Model
import data_set
from data_set import load_embeddings, fetch_data, fetch_embeddings
from data_set import MAX_SEQUENCE_LENGTH_HEAD
from data_set import MAX_SEQUENCE_LENGTH_BODY
from data_set import EMBEDDING_DIM
from data_set import VALIDATION_SPLIT
from data_set import MAX_NUM_WORDS, embeddings_index, labels_index
import numpy as np
from keras.utils import to_categorical
import pickle

In [2]:
load_embeddings()
(headlines, bodies_map, labels) = fetch_data()


Found 400000 word vectors.
All done


In [3]:
(data_head, data_body, embedding_layer_head, embedding_layer_body) = fetch_embeddings(headlines, bodies_map)

Preparing embedding matrix.
All done


In [4]:
pickle.dump((headlines, bodies_map, labels), open("train_file1.txt", "wb"))

In [5]:
pickle.dump((data_head, data_body, embedding_layer_head, embedding_layer_body), open("train_file2.txt", "wb"))

In [6]:
(headlines, bodies_map, _) = pickle.load(open("train_file1.txt", "rb"))

In [7]:
(data_head, data_body, embedding_layer_head, embedding_layer_body) = pickle.load(open("train_file2.txt", "rb"))

In [8]:
numLSTM = 200
outputClasses = 4
inputTitle = Input(shape=(MAX_SEQUENCE_LENGTH_HEAD, ), dtype='int32', name='input_title')
inputBody = Input(shape=(MAX_SEQUENCE_LENGTH_BODY, ), dtype='int32', name='input_body')

embedded_sequence_head = embedding_layer_head(inputTitle)
embedded_sequence_body = embedding_layer_body(inputBody)

titleLSTM = Bidirectional(LSTM(numLSTM, return_sequences = False))
bodyLSTM = Bidirectional(LSTM(numLSTM, return_sequences = False))

output1 = titleLSTM(embedded_sequence_head)
output2 = bodyLSTM(embedded_sequence_body)

concatTitleBody = Merge(mode="concat",name="concat_layer")([output1, output2])

hiddenDense = Dense(100, activation = 'relu')
dropout = Dropout(0.1)

outputDense = Dense(outputClasses, activation = 'softmax', name = 'out')

concat = hiddenDense(concatTitleBody)
concat = dropout(concat)
out = outputDense(concat)
# out.get_config()
# X = np.zeros(10, )

model = Model(inputs = [inputTitle, inputBody], outputs = [out])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_title (InputLayer)        (None, 50)           0                                            
__________________________________________________________________________________________________
input_body (InputLayer)         (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 50)       194000      input_title[0][0]                
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 200, 50)      1000000     input_body[0][0]                 
__________________________________________________________________________________________________
bidirectio

C:\Users\rachu\AppData\Local\conda\conda\envs\tf-cpu\lib\site-packages\ipykernel\__main__.py:15: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [9]:
# Compile

optimizer = Adam(lr=0.001, beta_1=0.99, beta_2=0.999, epsilon=1e-08) # Optimization hyperparameters.

model.compile(optimizer=optimizer,
              loss={'out':'categorical_crossentropy'},
              loss_weights={'out': 1.0}, # These can be tuned.
              metrics=['accuracy'])

In [10]:
len(labels)

0

In [11]:
# Fitting
from keras.utils import to_categorical
labels = to_categorical(np.asarray(list(zip(*headlines))[2]))
model.fit([data_head, data_body], labels, epochs=10) #, batch_size=500)
# validation_data=([data_head, data_body], labels),
model.save_weights('32b_10e.txt')

Epoch 1/10
49972/49972 [==============================] - 1675s 34ms/step - loss: 0.6801 - acc: 0.7499
Epoch 2/10
49972/49972 [==============================] - 1698s 34ms/step - loss: 0.4365 - acc: 0.8308
Epoch 3/10
49972/49972 [==============================] - 1725s 35ms/step - loss: 0.2790 - acc: 0.8937
Epoch 4/10
49972/49972 [==============================] - 1743s 35ms/step - loss: 0.1887 - acc: 0.9298
Epoch 5/10
49972/49972 [==============================] - 1765s 35ms/step - loss: 0.1278 - acc: 0.9522
Epoch 6/10
49972/49972 [==============================] - 1806s 36ms/step - loss: 0.0979 - acc: 0.9634
Epoch 7/10
49972/49972 [==============================] - 1820s 36ms/step - loss: 0.0744 - acc: 0.9725
Epoch 8/10
49972/49972 [==============================] - 1851s 37ms/step - loss: 0.0604 - acc: 0.9768
Epoch 9/10
49972/49972 [==============================] - 1890s 38ms/step - loss: 0.0533 - acc: 0.9806
Epoch 10/10
49972/49972 [==============================] - 1906s 38ms/ste

In [17]:
#Confusion matrix

# Predicting the Test set results
y_pred = model.predict([data_head, data_body])

lab = np.argmax(labels, axis = 1)
pred = np.argmax(y_pred, axis = 1)
from sklearn.metrics import confusion_matrix
confusion_matrix(lab, pred)


array([[36416,    47,     6,    76],
       [   26,  3604,    38,    10],
       [    7,    42,   790,     1],
       [   72,    35,     3,  8799]], dtype=int64)

In [47]:
y_pred.shape

(49972, 4)

In [15]:
import pydot

In [16]:
print pydot.find_graphviz()

SyntaxError: invalid syntax (<ipython-input-16-1da3c0c46f21>, line 1)

In [34]:
(test_headlines, test_bodies_map, test_labels) = fetch_data(mode = 'test')

test
All done


In [36]:
(test_data_head, test_data_body, test_embedding_layer_head, test_embedding_layer_body) = fetch_embeddings(test_headlines, test_bodies_map)

Preparing embedding matrix.
All done


In [37]:
test_data_head.shape

(25413, 50)

In [38]:
pickle.dump((test_headlines, test_bodies_map, test_labels), open("test_file1.txt", "wb"))

In [39]:
pickle.dump((test_data_head, test_data_body, test_embedding_layer_head, test_embedding_layer_body), open("test_file2.txt", "wb"))

In [40]:
(test_headlines, test_bodies_map, _) = pickle.load(open("test_file1.txt", "rb"))

In [41]:
(test_data_head, test_data_body, test_embedding_layer_head, test_embedding_layer_body) = pickle.load(open("test_file2.txt", "rb"))

In [42]:
#Confusion matrix

# Predicting the Test set results
test_y_pred = model.predict([test_data_head, test_data_body])

test_lab = np.argmax(test_labels, axis = 1)
pred = np.argmax(test_y_pred, axis = 1)
from sklearn.metrics import confusion_matrix
confusion_matrix(test_lab, test_pred)


ValueError: axis(=1) out of bounds

In [52]:
test_labels = to_categorical(np.asarray(list(zip(*test_headlines))[2]))
test_lab = np.argmax(test_labels, axis = 1)
pred = np.argmax(test_y_pred, axis = 1)
from sklearn.metrics import confusion_matrix
confusion_matrix(test_lab, pred)

array([[14955,   732,    94,  2568],
       [ 1559,    93,    14,   237],
       [  568,    31,     5,    93],
       [ 3452,   206,    25,   781]], dtype=int64)

In [56]:
np.sum(test_y_pred, axis = 0)

array([ 20151.82617188,   1202.01049805,    196.79116821,   3862.57177734], dtype=float32)

In [18]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.